# MNAV - Simple GNN

Dataset: MNAV

Modelo: GNN simple

## Importar Datos

In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from itertools import combinations
from copy import deepcopy
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
import torch
import networkx as nx
from sklearn.preprocessing import OrdinalEncoder

from torch_geometric.data import Data
from torch_geometric.nn.conv.dna_conv import Linear
from torch_geometric.utils import to_networkx, is_undirected, to_undirected
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ChebConv, SAGEConv, TAGConv, GraphConv
from torch_geometric.loader import DataLoader

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

El dataset se encuentra disponible en https://github.com/ffedee7/posifi_mnav/tree/master/data_analysis. El dataset disponible está anonimizado pero se puede deshacer con el código de ese repositorio.

In [ ]:
dataset = 'datos2.csv'
df = pd.read_csv(dataset)

In [ ]:
print(df.shape)
df.describe()

In [ ]:
df.head()

## Preprocesamiento

Se definen los APs que se quieren usar tanto para la construcción del grafo como para el modelo.

En este caso usamos solamente los APs que se encuentran dentro del MNAV, es decir que descartamos los APs que aparecen en las medidas pero que no son los que se instalaron.

In [ ]:
APs_MAC_2_4 = ['wifi-dc:a5:f4:43:85:c0',
'wifi-dc:a5:f4:43:27:e0',
'wifi-f8:4f:57:ab:da:00',
'wifi-5c:a4:8a:4c:05:c0',
'wifi-1c:1d:86:ce:ef:b0',
'wifi-dc:a5:f4:43:79:20',
'wifi-c0:7b:bc:36:9e:10',
'wifi-1c:1d:86:9f:99:20',
'wifi-c0:7b:bc:36:af:40',
'wifi-c0:7b:bc:36:af:80',
'wifi-1c:1d:86:b6:ac:80',
'wifi-dc:a5:f4:43:72:e0',
'wifi-f8:4f:57:ab:d8:60',
'wifi-dc:a5:f4:43:72:90',
'wifi-f8:4f:57:ab:ce:20']

APs_MAC_5 = ['wifi-dc:a5:f4:45:85:b0',
'wifi-dc:a5:f4:45:27:e0',
'wifi-f8:4f:57:ad:d9:60',
'wifi-5c:a4:8a:4e:05:30',
'wifi-1c:1d:86:d0:ef:00',
'wifi-dc:a5:f4:45:79:10',
'wifi-c0:7b:bc:38:9e:00',
'wifi-1c:1d:86:a1:99:00',
'wifi-c0:7b:bc:38:af:30',
'wifi-c0:7b:bc:38:af:70',
'wifi-1c:1d:86:b8:ac:80',
'wifi-dc:a5:f4:45:72:d0',
'wifi-f8:4f:57:ad:d7:c0',
'wifi-dc:a5:f4:45:72:80',
'wifi-f8:4f:57:ad:cd:80']

In [ ]:
def preprocess_dataset(dataset_path, dataset_percentage=None):
    df = pd.read_csv(dataset_path)
    # paso los NaN a 0
    df = df.fillna(0) 

    # sumo 100 a los valores de RSSI y ahora 0 es el minimo
    df.iloc[:,1:] = 100 + df.iloc[:,1:]
    values = df.iloc[:,1:]

    # las medidas originales en 0 las asumo como que estaban muy lejos
    # entonces las dejo en 0 que es el nuevo valor minimo
    values[values==100] = 0 
    df.iloc[:,1:] = values

    # armo dos datsets: uno con las medidas solamente de la frecuencia
    # 2.4GHz y otro con las frecuencias 2.4GHz y 5GHz
    data_2_4 = df[['location'] + APs_MAC_2_4] # REVISAR PORQUE CREO QUE NO LO VUELVO A USAR
    data_2_4_5 = df[['location'] + APs_MAC_2_4 + APs_MAC_5]

    if dataset_percentage:
        data_2_4_5 = data_2_4_5.sample(frac=dataset_percentage)     
    
    # paso las zonas por un ordinal encoder
    enc = OrdinalEncoder(dtype=int)
    y = enc.fit_transform(data_2_4_5['location'].values.reshape(-1,1))
    X = data_2_4_5.iloc[:,1:].values

    # print(enc.categories_)

    # separo el dataset en train y test 80-20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

    print("X_train shape: ", X_train.shape)
    return X_train, X_test, y_train, y_test

## Grafo

En las siguientes celdas se describe un poco el dataset y se muestran las distribuciones de potencia por AP.

In [ ]:
def graph_creator(X_G, th=10, cols=None):
    """
    Dado un dataset y un threshold se arma un grafo basado en las medidas de RRSI
    """

    columns = cols if cols else ['AP1', 'AP2', 'AP3', 'AP4', 'AP5', 'AP6', 'AP7', 'AP8', 'AP9', 'AP10', 'AP11', 'AP12', 'AP13', 'AP14', 'AP15']
    df_data_train = pd.DataFrame(X_G, columns=columns)
    df_G = pd.DataFrame(columns = ['from', 'to', 'weight']) 

    for ap in columns:
        # para cada AP me quedo con las instancias donde el RSSI esta en el rango
        # (max-th) intentando estimar las instancias mas cercanas al AP
        max_val = df_data_train[ap].max()
        df_aux_i = df_data_train[df_data_train[ap]  > (max_val - th)]
        df_aux_i = df_aux_i.drop(ap, axis=1) 
        df_aux_i.head()

        for k, v in df_aux_i.mean().items():
            # armo las aristas con el promedio de RSSI que ven las instancias 
            # filtradas al resto de los APs
            # weight = v
            # if df_G.loc[(df_G['from'] == k) & (df_G['to'] == ap)].weight.any():
            #     weight = np.mean([float(df_G.loc[(df_G['from'] == k) & (df_G['to'] == ap)].weight), weight])
            #     df_G.loc[(df_G['from'] == k) & (df_G['to'] == ap)] = k, ap, weight
            df_G = df_G.append({'from':ap, 'to': k, 'weight': v}, ignore_index=True)
        

    edge_index_first_row = []
    edge_index_second_row = []
    edge_attr = []
    for index, row in df_G.iterrows():
        edge_index_first_row.append(columns.index(row['from']))
        edge_index_second_row.append(columns.index(row['to']))
        edge_attr.append([float(row.weight)])

    
    edge_index = torch.tensor([edge_index_first_row, edge_index_second_row], dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)                           
    edge_index, edge_attr = to_undirected(edge_index, edge_attr, reduce="mean")
    return edge_index, edge_attr

In [ ]:
def build_dataset(X, y, graph):
    dataset = []
    for i in range(len(y)):
        data = deepcopy(graph)
        data.x = torch.Tensor(X[i])
        data.y = torch.Tensor(y[i])
        data.train_mask = torch.Tensor([True]*len(y))
        data.val_mask = torch.Tensor([True]*len(y))
        data.test_mask = torch.Tensor([True]*len(y))                
        dataset.append(data)
    return dataset

## Modelo

In [ ]:
class GNN_GCNConv(torch.nn.Module):
    def __init__(self, conv_out_features: list = [16, 20]):
        super().__init__()
        self.conv_out_features = conv_out_features
        self.conv1 = GCNConv(2, self.conv_out_features[0], bias=True, normalize=True)
        self.conv2 = GCNConv(self.conv_out_features[0], self.conv_out_features[1], bias=True, normalize=True)
        self.fc = torch.nn.Linear(self.conv_out_features[-1]*15,16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # print("After Conv1: ", x.shape)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)        
        # print("After Conv2: ", x.shape)

        # x = torch.flatten(x, 0)
        x = torch.reshape(x, (int(x.shape[0]/15),self.conv_out_features[-1]*15))
        # print("After Flatten: ", x.shape)
        
        x = self.fc(x)
        x = F.relu(x)        
        # print("After FC: ", x.shape)

        return x

In [ ]:
class GNN_TAGConv(torch.nn.Module):
    def __init__(self, k: list = [1,1], conv_out_features: list = [16, 20]):
        super().__init__()
        self.k = k
        self.conv_out_features = conv_out_features
        self.conv1 = TAGConv(2, self.conv_out_features[0], K=self.k[0], bias=True, normalize=True)
        self.conv2 = TAGConv(self.conv_out_features[0], self.conv_out_features[1], K=self.k[1], bias=True, normalize=True)
        self.fc = torch.nn.Linear(self.conv_out_features[-1]*15,16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # print("After Conv1: ", x.shape)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)        
        # print("After Conv2: ", x.shape)

        # x = torch.flatten(x, 0)
        x = torch.reshape(x, (int(x.shape[0]/15),self.conv_out_features[-1]*15))
        # print("After Flatten: ", x.shape)
        
        x = self.fc(x)
        x = F.relu(x)        
        # print("After FC: ", x.shape)

        return x

In [ ]:
class GNN_GraphConv(torch.nn.Module):
    def __init__(self, aggr = "add", conv_out_features: list = [16, 20]):
        super().__init__()
        self.aggr = aggr
        self.conv_out_features = conv_out_features
        self.conv1 = GraphConv(2, self.conv_out_features[0], aggr=self.aggr, bias=True)
        self.conv2 = GraphConv(self.conv_out_features[0], self.conv_out_features[1], aggr=self.aggr, bias=True)
        self.fc = torch.nn.Linear(self.conv_out_features[-1]*15,16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # print("After Conv1: ", x.shape)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)        
        # print("After Conv2: ", x.shape)

        # x = torch.flatten(x, 0)
        x = torch.reshape(x, (int(x.shape[0]/15),self.conv_out_features[-1]*15))
        # print("After Flatten: ", x.shape)
        
        x = self.fc(x)
        # x = F.relu(x)        
        # print("After FC: ", x.shape)

        return x

## Entrenamiento

In [ ]:
plt.hist(data.edge_attr)

In [ ]:
# data.edge_attr = (data.edge_attr - data.edge_attr.mean()) / data.edge_attr.std()
data.edge_attr = (data.edge_attr - data.edge_attr.min()) / data.edge_attr.max()
data.edge_attr *= 4
# data.edge_attr = torch.nn.functional.normalize(data.edge_attr, dim=0)

In [ ]:
g = to_networkx(data, edge_attrs=["edge_attr"], to_undirected=True)
weights = nx.get_edge_attributes(g,'edge_attr').values()
# pos = nx.circular_layout(g)
graph = nx.draw(g, width=list(weights), with_labels=True, node_color='black', font_color="white", font_family='Helvetica')
plt.savefig("mnav_graph.pdf")

In [ ]:
# Split de los datos y armado del objeto Data con el grafo

X_train, X_test, y_train, y_test = preprocess_dataset(dataset)
edge_index, edge_attr = graph_creator(X_train[:,:15], th=10) #el grafo lo armo solo con los datos de 2.4Ghz
data = Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=15)
print(f"Undirected: {data.is_undirected()}")

In [ ]:
# Armado del dataset

x_training_data = np.reshape(X_train,(X_train.shape[0],15,2))
x_test_data = np.reshape(X_test,(X_test.shape[0],15,2))
y_training_data = y_train
y_test_data = y_test

#normalize (x-mean)/std
mean = x_training_data.mean(axis=0)
std = x_training_data.std(axis=0)

x_training_data = x_training_data - mean
x_training_data /= std
x_test_data = x_test_data - mean
x_test_data /= std

train_dataset = build_dataset(x_training_data, y_training_data, data)
test_dataset = build_dataset(x_test_data, y_test_data, data)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
batch_size = 16
learning_rate = 0.001

### GCNConv 

In [ ]:
model = GNN_GCNConv(conv_out_features=[20,20]).to(device)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
loss = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


In [ ]:
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

m = torch.nn.Softmax(dim=1)

for epoch in range(100):
    print(f"Epoch: {epoch}")
    
    # TRAIN
    model.train()
    train_accuracy_epoch = []
    train_loss_epoch = []
    for data in train_loader:
        import ipdb; ipdb.set_trace()
        optimizer.zero_grad()
        # print(data.x.shape)
        # print(data.y.shape)
        
        
        out = model(data.to(device))
        # print(out)
        # out_softmax = np.array(torch.argmax(out, dim=0)).item()
        # out_softmax = torch.tensor([out_softmax])
        loss_result = loss(out, data.y.type(torch.long))        
        loss_result.backward()
        
        train_loss_epoch.append(loss_result.detach().cpu())
        output = m(out)
        train_accuracy_epoch.append(accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))


        optimizer.step()

    # if scheduler.get_last_lr()[0] > 0.0005:
    #     scheduler.step()

    train_accuracy.append(np.mean(train_accuracy_epoch))
    train_loss.append(np.mean(train_loss_epoch))

    # VALIDATION
    model.eval()
    test_accuracy_epoch = []
    test_loss_epoch = []
    for data in test_loader:
        out = model(data.to(device))
        loss_result = loss(out, data.y.type(torch.long))        
        
        test_loss_epoch.append(loss_result.detach().cpu())
        output = m(out)
        test_accuracy_epoch.append(accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))

    test_accuracy.append(np.mean(test_accuracy_epoch))
    test_loss.append(np.mean(test_loss_epoch))


print(f"Last LR: {scheduler.get_last_lr()}")

plt.figure()
plt.plot(train_loss, label="Train loss")
plt.plot(test_loss, label="Validation loss")
plt.legend()

plt.figure()
plt.plot(train_accuracy, label="Train accuracy")
plt.plot(test_accuracy, label="Validation accuracy")
plt.legend()

plt.show()


In [ ]:
m = torch.nn.Softmax(dim=1)
output = m(out)
accuracy = accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1)))

print(accuracy)
print(classification_report(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))

### TAGConv 

In [ ]:
model = GNN_TAGConv(k=[2,2], conv_out_features=[20,20]).to(device)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
loss = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

In [ ]:
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []
best_test_accuracy = 0

m = torch.nn.Softmax(dim=1)

for epoch in range(50):
    print(f"Epoch: {epoch}")
    
    # TRAIN
    model.train()
    train_accuracy_epoch = []
    train_loss_epoch = []
    for data in train_loader:

        optimizer.zero_grad()
        # print(data.x.shape)
        # print(data.y.shape)
        
        
        out = model(data.to(device))
        # print(out)
        # out_softmax = np.array(torch.argmax(out, dim=0)).item()
        # out_softmax = torch.tensor([out_softmax])
        loss_result = loss(out, data.y.type(torch.long))        
        loss_result.backward()
        
        train_loss_epoch.append(loss_result.detach().cpu())
        output = m(out)
        train_accuracy_epoch.append(accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))


        optimizer.step()
    
    if (epoch+1)%10 == 0:
        scheduler.step()

    train_accuracy.append(np.mean(train_accuracy_epoch))
    train_loss.append(np.mean(train_loss_epoch))

    # VALIDATION
    model.eval()
    test_accuracy_epoch = []
    test_loss_epoch = []
    for data in test_loader:
        out = model(data.to(device))
        loss_result = loss(out, data.y.type(torch.long))        
        
        test_loss_epoch.append(loss_result.detach().cpu())
        output = m(out)
        test_accuracy_epoch.append(accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))

    test_accuracy.append(np.mean(test_accuracy_epoch))
    if test_accuracy[-1] > best_test_accuracy:
        best_test_accuracy = test_accuracy[-1]        
        torch.save(model.state_dict(), "MNAV_best_model.pth")
        
    test_loss.append(np.mean(test_loss_epoch))
    print(f"    Train Loss {np.mean(train_loss_epoch)}, Val Loss {np.mean(test_loss_epoch)}")


print(f"Last LR: {scheduler.get_last_lr()}")
print(f"Best Accuracy: Train {np.max(train_accuracy)}, Val {np.max(test_accuracy)}")

plt.figure()
plt.plot(train_loss, label="Train loss")
plt.plot(test_loss, label="Validation loss")
plt.legend()

plt.figure()
plt.plot(train_accuracy, label="Train accuracy")
plt.plot(test_accuracy, label="Validation accuracy")
plt.legend()

plt.show()



In [ ]:
m = torch.nn.Softmax(dim=1)
output = m(out)
accuracy = accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1)))

print(accuracy)
print(classification_report(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))

### GraphConv 

In [ ]:
model = GNN_GraphConv(aggr="mean", conv_out_features=[20,20]).to(device)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
loss = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


In [ ]:
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

m = torch.nn.Softmax(dim=1)

for epoch in range(100):
    print(f"Epoch: {epoch}")
    
    # TRAIN
    model.train()
    train_accuracy_epoch = []
    train_loss_epoch = []
    for data in train_loader:

        optimizer.zero_grad()
        # print(data.x.shape)
        # print(data.y.shape)
        
        
        out = model(data.to(device))
        # print(out)
        # out_softmax = np.array(torch.argmax(out, dim=0)).item()
        # out_softmax = torch.tensor([out_softmax])
        
        import ipdb; ipdb.set_trace()
        
        loss_result = loss(out, data.y.type(torch.long))        
        loss_result.backward()
        
        train_loss_epoch.append(loss_result.detach().cpu())
        output = m(out)
        train_accuracy_epoch.append(accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))


        optimizer.step()

    # if scheduler.get_last_lr()[0] > 0.0005:
    #     scheduler.step()

    train_accuracy.append(np.mean(train_accuracy_epoch))
    train_loss.append(np.mean(train_loss_epoch))

    # VALIDATION
    model.eval()
    test_accuracy_epoch = []
    test_loss_epoch = []
    for data in test_loader:
        out = model(data.to(device))
        loss_result = loss(out, data.y.type(torch.long))        
        
        test_loss_epoch.append(loss_result.detach().cpu())
        output = m(out)
        test_accuracy_epoch.append(accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))

    test_accuracy.append(np.mean(test_accuracy_epoch))
    test_loss.append(np.mean(test_loss_epoch))


print(f"Last LR: {scheduler.get_last_lr()}")

plt.figure()
plt.plot(train_loss, label="Train loss")
plt.plot(test_loss, label="Validation loss")
plt.legend()

plt.figure()
plt.plot(train_accuracy, label="Train accuracy")
plt.plot(test_accuracy, label="Validation accuracy")
plt.legend()

plt.show()


In [ ]:
m = torch.nn.Softmax(dim=1)
output = m(out)
accuracy = accuracy_score(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1)))

print(accuracy)
print(classification_report(data.y.cpu().reshape(-1).type(torch.long), np.array(torch.argmax(output.cpu(), axis=1))))

## Análisis variando cantidad de muestras

### KNN

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:1')
batch_size = 16
learning_rate = 0.01  
print_every = 5

porcentajes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
accuracy = {"0.1":[], "0.2":[], "0.3":[], "0.4":[], "0.5":[], "0.6":[], "0.7":[], "0.8":[], "0.9":[], "1":[]}

for porc in porcentajes:
    print('Porcentaje de datos: ', porc)
    
    for i in range(10):    

        X_train, X_test, y_train, y_test = preprocess_dataset(dataset, dataset_percentage=porc)

        # x_training_data = np.reshape(X_train,(X_train.shape[0],15,2))
        # x_test_data = np.reshape(X_test,(X_test.shape[0],15,2))
        y_training_data = y_train
        y_test_data = y_test

        #normalize (x-mean)/std
        mean = X_train.mean(axis=0)
        std = X_train.std(axis=0)

        x_training_data = X_train - mean
        x_training_data /= std
        x_test_data = X_test - mean
        x_test_data /= std

        from sklearn.neighbors import KNeighborsClassifier
        from sklearn.metrics import accuracy_score, classification_report     
        
        neigh = KNeighborsClassifier(n_neighbors=3)
        neigh.fit(x_training_data, y_training_data)
        y_pred_knn = neigh.predict(x_test_data)
        acc = accuracy_score(y_test_data, y_pred_knn)

        print(f"Accuracy: {acc}")
        accuracy[str(porc)].append(acc)


In [ ]:
X_test.shape

In [ ]:
accuracy